In [27]:
import tensorflow as tf

v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

v1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
v2 = tf.constant([[5.0, 6.0], [7.0, 8.0]])

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print "Tensor V value : \n", sess.run(v), "\n"
    print "Tensor V1 value : \n", sess.run(v1), "\n"
    print "Tensor V2 value : \n", sess.run(v2), "\n"

Tensor V value : 
[[1. 2. 3.]
 [4. 5. 6.]] 

Tensor V1 value : 
[[1. 2.]
 [3. 4.]] 

Tensor V2 value : 
[[5. 6.]
 [7. 8.]] 



In [30]:
import tensorflow as tf

v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

v1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
v2 = tf.constant([[5.0, 6.0], [7.0, 8.0]])
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    print "Tensor V Log : \n", sess.run(tf.log(v)), "\n"
    print "Tensor V reduce mean :\n", sess.run(tf.reduce_mean(v)), "\n"

Tensor V Log : 
[[0.        0.6931472 1.0986123]
 [1.3862944 1.609438  1.7917595]] 

Tensor V reduce mean :
3.5 



In [29]:
import tensorflow as tf

v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
v1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
v2 = tf.constant([[5.0, 6.0], [7.0, 8.0]])

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print "V1 * V2 : \n", sess.run(v1 * v2), "\n"
    print "V1 matmul V2 : \n", sess.run(tf.matmul(v1, v2)), "\n"
    print "clip_by_value : \n", sess.run(tf.clip_by_value(v, 2.5, 4.5))

V1 * V2 : 
[[ 5. 12.]
 [21. 32.]] 

V1 matmul V2 : 
[[19. 22.]
 [43. 50.]] 

clip_by_value : 
[[2.5 2.5 3. ]
 [4.  4.5 4.5]]


In [33]:
import tensorflow as tf
v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])

sess = tf.InteractiveSession()
print sess.run(tf.greater(v1, v2))


[False False  True  True]


In [44]:
import tensorflow as tf
import numpy as np
from numpy.random import RandomState

batch_size = 8

x = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y-input')

w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

loss_less = 10
loss_more = 1

loss = tf.reduce_mean(tf.where(tf.greater(y, y_), (y-y_) * loss_more, (y_ - y) * loss_less))
opt = tf.train.AdamOptimizer(0.001).minimize(loss)


rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = np.array([[x1 + x2 + rdm.rand()/10.0 - 0.05] for (x1, x2) in X])

print X.shape, Y.shape
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(opt, feed_dict={x: X[start: end], y_: Y[start: end]})
        if i % 1000 == 0:
            print "\nparameter w1: \n", sess.run(w1)
            print "loss:", sess.run(loss, feed_dict={x: X, y_: Y})


(128, 2) (128, 1)



parameter w1: 
[[-0.81031823]
 [ 1.4855988 ]]
loss: 5.9678044



parameter w1: 
[[0.01247112]
 [2.1385448 ]]
loss: 1.3432351



parameter w1: 
[[0.45567414]
 [2.1706066 ]]
loss: 0.6813779



parameter w1: 
[[0.69968724]
 [1.8465308 ]]
loss: 0.44227606



parameter w1: 
[[0.89886665]
 [1.2973602 ]]
loss: 0.16296774


In [45]:
import tensorflow as tf


def get_weight(shape, _lambda):
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(_lambda)(var))
    return var


x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))

batch_size = 8

layer_dimension = [2, 10, 10, 10, 1]

n_layers = len(layer_dimension)

cur_layer = x
in_dimension = layer_dimension[0]

for i in range(1, n_layers):
    out_dimension = layer_dimension[i]
    
    weight = get_weight([in_dimension, out_dimension], 0.001)
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))
    
    cur_layer = tf.nn.relu(tf.matmul(cur_layer, weight) + bias)
    in_dimension = layer_dimension[i]

mse_loss = tf.reduce_mean(tf.square(y_ - cur_layer))
tf.add_to_collection('losses', mse_loss)

loss = tf.add_n(tf.get_collection('losses'))

In [50]:
import tensorflow as tf

v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

ema = tf.train.ExponentialMovingAverage(0.99, step)
maintain_averages_op = ema.apply([v1])

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    print sess.run([v1, ema.average(v1)]), '\n'
    print sess.run(tf.assign(v1, 5)), '\n'
    
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)]), '\n'
    
    sess.run(tf.assign(step, 10000))
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)]), '\n'
    
    sess.run(maintain_averages_op)
    print sess.run([v1, ema.average(v1)]), '\n'

[0.0, 0.0] 

5.0 

[5.0, 4.5] 



[10.0, 4.555] 

[10.0, 4.60945] 

